In [1]:
import scanpy as sc
import anndata as ann
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy.sparse import issparse
from tqdm import tqdm

from sklearn.decomposition import PCA
import scvelo as scv
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import clear_output

from unitvelo.eval_utils import cross_boundary_correctness, inner_cluster_coh

(Running UniTVelo 0.2.5.2)
2023-02-07 11:02:30


In [2]:
import sys
sys.path.insert(0, '../../../git/velocity_package/')
import velocity


In [10]:
def kvelo_pipeline(adata0, name):
    adata = adata0.copy()
    scv.pp.moments(adata, n_pcs=50, n_neighbors=30)
    velocity.tl.fit.recover_reaction_rate_pars(adata, use_raw=False)
    velocity.tl.fit.get_velocity(adata, use_raw=False, normalise=None)
    
    V = adata.layers['velocity']
    V = np.nan_to_num(V, nan=0, neginf=0, posinf=0)
    return V

def kvelo_pipeline_without_smoothing(adata0, name):
    adata = adata0.copy()
    fix = lambda x: x.A if issparse(x) else x
    adata.layers['unspliced'] = fix(adata.layers['unspliced'])
    adata.layers['spliced'] = fix(adata.layers['spliced'])
    velocity.tl.fit.recover_reaction_rate_pars(adata, use_raw=True)
    velocity.tl.fit.get_velocity(adata, use_raw=True, normalise=None)
    
    V = adata.layers['velocity']
    V = np.nan_to_num(V, nan=0, neginf=0, posinf=0)
    return V

In [11]:
## functions used in preparing data for benchmarking

def project_to_pca(adata):
    X = adata.layers['total']
    V = adata.layers['velocity']

    X = np.array(X.A if issparse(X) else X)
    V = np.array(V.A if issparse(V) else V)
    V = np.nan_to_num(V, nan=0, neginf=0, posinf=0)
    Y = np.clip(X + V, 0, 1000)


    Xlog = np.log1p(X)
    pca = PCA()
    Xpca = pca.fit_transform(Xlog)

    Ylog = np.log1p(Y)
    Ypca = pca.transform(Ylog)
    V = Ypca - Xpca
    return V

def prepare_for_test(
    adata,
    name,
    func,
    ndims=50,
    pt=True,
):
    x_pca = adata.obsm['X_pca']
    velocity = func(adata, name)

    test = ann.AnnData(X=adata.X, obs=adata.obs, var=adata.var,
                       layers={'total':adata.layers['total'],
                               'velocity':velocity})

    test.obsm['X_pca'] = x_pca[:,:ndims]
    test.obsm['cellrank_baseline'] = adata.obsm['velocity_cr_pca'][:,:ndims]
    if pt:
        test.obsm['pseudotime_baseline'] = adata.obsm['velocity_pst'][:,:ndims]
    else:
        ## this is a lazy implementation, will create meaningless comparison
        ## but it will never get saved
        ## this is just for the maxi dataset that we don't have a good
        ## pseudotime trajectory skeleton for.
        test.obsm['pseudotime_baseline'] = np.zeros_like(test.obsm['cellrank_baseline'])
        
    test.obsm['velocity_pca'] = project_to_pca(test)[:,:ndims]
    
    scv.pp.neighbors(test)
    return test


In [12]:
# the object that will contain the data and data-specific parameters for benchmarking

class BenchMarkingData:
    def __init__(self, name, func, pt=True):
        self.name = name
        adata = sc.read_h5ad(f'../data/benchmarking_data/{name}.h5ad')

        self.adata = prepare_for_test(
            adata,
            name,
            func,
            pt=pt
        )
        
        self.cluster_edges()
        
    def cluster_edges(self):
        if "mini_V3" in self.name:
            self.obs = 'leiden'
            self.cluster_edges = [
                ('5','14'),
                ('14','8'),
                ('8','21')
            ]
        elif "mini_MN" in self.name:
            self.obs = 'leiden'
            self.cluster_edges = [
                ('16','15'),
                ('20','23'),
                ('13','18')
            ]
        elif "mini_MD" in self.name:
            self.obs = 'leiden'
            self.cluster_edges = [
                ('9','12'),
                ('25','4'),
                ('4','6'),
                ('6','22')
            ]
        elif "midi_NM" in self.name:
            self.obs = 'cell_annotation'
            self.cluster_edges = [
                ('Early_Neural','Neural'),
                ('NMP','Early_Neural'),
                ('NMP','Mesoderm')
            ]
        elif 'midi_Ne' in self.name:
            self.obs = 'cell_annotation'
            self.cluster_edges = [
                ('Neural','pMN'),
                ('pMN','MN'),
                ('pMN','p3'),
                ('p3','V3')
            ]
        elif 'maxi' in self.name:
            self.obs = 'cell_annotation'
            self.cluster_edges = [
                ('Early_Neural','Neural'),
                ('NMP','Early_Neural'),
                ('NMP','Mesoderm'),
                ('Neural','pMN'),
                ('pMN','MN'),
                ('pMN','p3'),
                ('p3','V3')
            ]

In [13]:
### the benchmarking functions
### cross_boundary_correctness and inner_cluster_coh use implementation from unitvelo
### web: https://unitvelo.readthedocs.io/en/latest/
### paper: https://www.nature.com/articles/s41467-022-34188-7

def baseline_scores(
    adata
):
    X = adata.obsm['velocity_pca']
    Y1 = adata.obsm['cellrank_baseline']
    Y2 = adata.obsm['pseudotime_baseline']
    cr_scores = np.diagonal(cosine_similarity(X, Y1))
    pt_scores = np.diagonal(cosine_similarity(X, Y2))
    return cr_scores, pt_scores

def run_tests(bm):
    cbd = cross_boundary_correctness(
        bm.adata,
        k_cluster=bm.obs,
        k_velocity='velocity',
        x_emb='X_pca',
        cluster_edges=bm.cluster_edges
    )[1]

    icc = inner_cluster_coh(
        bm.adata,
        k_cluster=bm.obs,
        k_velocity='velocity',
    )[1]
    
    crs, pts = baseline_scores(bm.adata)
    
    return cbd, icc, crs, pts

        

In [14]:
def perform_benchmark(
    pipeline_name,
    velocity_pipeline, 
    output_folder
):
    dataset = ['splicing_mini_V3', 'splicing_mini_MN', 'splicing_mini_MD',
               'splicing_midi_NM', 'splicing_midi_Ne', 'splicing_maxi']
    
    for ds in tqdm(dataset):  
        bm_data = BenchMarkingData(ds, velocity_pipeline, pt=(ds!='splicing_maxi'))
        print(ds)
        cbd, icc, crs, pts = run_tests(bm_data)
        np.save(f'{output_folder}/{ds}_{pipeline_name}_CBD.npy', cbd)
        np.save(f'{output_folder}/{ds}_{pipeline_name}_ICC.npy', icc)
        np.save(f'{output_folder}/{ds}_{pipeline_name}_CRS.npy', crs)
        if ds!='splicing_maxi':
            np.save(f'{output_folder}/{ds}_{pipeline_name}_PTS.npy', pts)

In [15]:
# ~4 hours per run

# perform_benchmark(
#     pipeline_name='kvelo',
#     velocity_pipeline=kvelo_pipeline, 
#     output_folder='../output_data/' 
# )

In [16]:
perform_benchmark(
    pipeline_name='kvelo_RAW',
    velocity_pipeline=kvelo_pipeline_without_smoothing, 
    output_folder='../output_data/'
)

  0%|          | 0/6 [00:00<?, ?it/s]

Optimization terminated successfully.
         Current function value: 35.857615
         Iterations: 369
         Function evaluations: 647
Optimization terminated successfully.
         Current function value: 40.184410
         Iterations: 260
         Function evaluations: 449
Optimization terminated successfully.
         Current function value: 32.840206
         Iterations: 508
         Function evaluations: 857
Optimization terminated successfully.
         Current function value: 36.889116
         Iterations: 984
         Function evaluations: 1634
Optimization terminated successfully.
         Current function value: 27.168626
         Iterations: 505
         Function evaluations: 857
Optimization terminated successfully.
         Current function value: 30.153232
         Iterations: 385
         Function evaluations: 662
Optimization terminated successfully.
         Current function value: 38.519015
         Iterations: 274
         Function evaluations: 474
Optimization

Optimization terminated successfully.
         Current function value: 26.606151
         Iterations: 270
         Function evaluations: 480
Optimization terminated successfully.
         Current function value: 35.976783
         Iterations: 269
         Function evaluations: 463
Optimization terminated successfully.
         Current function value: 33.339041
         Iterations: 457
         Function evaluations: 789
Optimization terminated successfully.
         Current function value: 32.014453
         Iterations: 914
         Function evaluations: 1589
Optimization terminated successfully.
         Current function value: 23.825141
         Iterations: 327
         Function evaluations: 564
Optimization terminated successfully.
         Current function value: 21.103545
         Iterations: 299
         Function evaluations: 520
Optimization terminated successfully.
         Current function value: 45.360524
         Iterations: 511
         Function evaluations: 860
Optimization

Optimization terminated successfully.
         Current function value: 32.082447
         Iterations: 471
         Function evaluations: 792
Optimization terminated successfully.
         Current function value: 34.062035
         Iterations: 452
         Function evaluations: 790
Optimization terminated successfully.
         Current function value: 32.630744
         Iterations: 288
         Function evaluations: 498
Optimization terminated successfully.
         Current function value: 38.961072
         Iterations: 486
         Function evaluations: 818
Optimization terminated successfully.
         Current function value: 29.777510
         Iterations: 308
         Function evaluations: 534
Optimization terminated successfully.
         Current function value: 40.746661
         Iterations: 344
         Function evaluations: 584
Optimization terminated successfully.
         Current function value: 32.822697
         Iterations: 240
         Function evaluations: 403
Optimization 

Optimization terminated successfully.
         Current function value: 31.240982
         Iterations: 325
         Function evaluations: 576
Optimization terminated successfully.
         Current function value: 38.791962
         Iterations: 650
         Function evaluations: 1117
Optimization terminated successfully.
         Current function value: 32.671802
         Iterations: 241
         Function evaluations: 421
Optimization terminated successfully.
         Current function value: 34.240746
         Iterations: 345
         Function evaluations: 572
Optimization terminated successfully.
         Current function value: 31.627809
         Iterations: 342
         Function evaluations: 599
Optimization terminated successfully.
         Current function value: 42.056151
         Iterations: 443
         Function evaluations: 757
Optimization terminated successfully.
         Current function value: 34.143193
         Iterations: 282
         Function evaluations: 492
Optimization

...

Optimization terminated successfully.
         Current function value: 35.400249
         Iterations: 352
         Function evaluations: 626
Optimization terminated successfully.
         Current function value: 26.156997
         Iterations: 282
         Function evaluations: 494
Optimization terminated successfully.
         Current function value: 34.010672
         Iterations: 354
         Function evaluations: 629
Optimization terminated successfully.
         Current function value: 38.328349
         Iterations: 411
         Function evaluations: 706
Optimization terminated successfully.
         Current function value: 28.381509
         Iterations: 248
         Function evaluations: 440
Optimization terminated successfully.
         Current function value: 33.873223
         Iterations: 411
         Function evaluations: 721
Optimization terminated successfully.
         Current function value: 31.343493
         Iterations: 421
         Function evaluations: 731
Optimization 

    finished (0:00:09) --> added 
    'distances' and 'connectivities', weighted adjacency matrices (adata.obsp)
splicing_maxi


100%|██████████| 6/6 [2:23:59<00:00, 1439.87s/it]


Optimization terminated successfully.
         Current function value: 23.131509
         Iterations: 385
         Function evaluations: 668
Optimization terminated successfully.
         Current function value: 32.353283
         Iterations: 500
         Function evaluations: 866
Optimization terminated successfully.
         Current function value: 31.021765
         Iterations: 278
         Function evaluations: 489
Optimization terminated successfully.
         Current function value: 30.140796
         Iterations: 405
         Function evaluations: 679
Optimization terminated successfully.
         Current function value: 29.325982
         Iterations: 484
         Function evaluations: 817
Optimization terminated successfully.
         Current function value: 32.248363
         Iterations: 436
         Function evaluations: 760
Optimization terminated successfully.
         Current function value: 29.418836
         Iterations: 255
         Function evaluations: 438
Optimization 

Optimization terminated successfully.
         Current function value: 32.888284
         Iterations: 431
         Function evaluations: 739
Optimization terminated successfully.
         Current function value: 27.164673
         Iterations: 372
         Function evaluations: 646
Optimization terminated successfully.
         Current function value: 38.951060
         Iterations: 516
         Function evaluations: 877
Optimization terminated successfully.
         Current function value: 41.938782
         Iterations: 259
         Function evaluations: 449
Optimization terminated successfully.
         Current function value: 32.441379
         Iterations: 300
         Function evaluations: 518
Optimization terminated successfully.
         Current function value: 38.058707
         Iterations: 246
         Function evaluations: 426
Optimization terminated successfully.
         Current function value: 31.172776
         Iterations: 369
         Function evaluations: 629
Optimization 

In [5]:
import os
import numpy as np

In [7]:
print(np.sort([f for f in os.listdir('../output_data/') if 'kvelo' in f]))

['splicing_maxi_kvelo_CBD.npy' 'splicing_maxi_kvelo_CRS.npy'
 'splicing_maxi_kvelo_ICC.npy' 'splicing_maxi_kvelo_RAW_CBD.npy'
 'splicing_maxi_kvelo_RAW_CRS.npy' 'splicing_maxi_kvelo_RAW_ICC.npy'
 'splicing_midi_NM_kvelo_CBD.npy' 'splicing_midi_NM_kvelo_CRS.npy'
 'splicing_midi_NM_kvelo_ICC.npy' 'splicing_midi_NM_kvelo_PTS.npy'
 'splicing_midi_NM_kvelo_RAW_CBD.npy' 'splicing_midi_NM_kvelo_RAW_CRS.npy'
 'splicing_midi_NM_kvelo_RAW_ICC.npy' 'splicing_midi_NM_kvelo_RAW_PTS.npy'
 'splicing_midi_Ne_kvelo_CBD.npy' 'splicing_midi_Ne_kvelo_CRS.npy'
 'splicing_midi_Ne_kvelo_ICC.npy' 'splicing_midi_Ne_kvelo_PTS.npy'
 'splicing_midi_Ne_kvelo_RAW_CBD.npy' 'splicing_midi_Ne_kvelo_RAW_CRS.npy'
 'splicing_midi_Ne_kvelo_RAW_ICC.npy' 'splicing_midi_Ne_kvelo_RAW_PTS.npy'
 'splicing_mini_MD_kvelo_CBD.npy' 'splicing_mini_MD_kvelo_CRS.npy'
 'splicing_mini_MD_kvelo_ICC.npy' 'splicing_mini_MD_kvelo_PTS.npy'
 'splicing_mini_MD_kvelo_RAW_CBD.npy' 'splicing_mini_MD_kvelo_RAW_CRS.npy'
 'splicing_mini_MD_kvelo_RAW